# "Human upgrade"

Project #5

*Lab Journal by Artem Vasilev and Tatiana Lisitsa*

---

## **Preparing**

Update packages:

`$ sudo apt update && sudo apt upgrade`

### **VE setup and downloading tools**

Create mamba VE:

`$ mamba create -n bi_practice_5 -c conda-forge python=3.12`

`$ mamba activate bi_practice_5`

Install open-cravat:

`$ pip install open-cravat`

`$ oc config md {path/to/dir/with/annotators}`  # optional

`$ oc module install-base`

`$ oc module ls -a -t annotator`  # see available annotators

`$ oc module install hg19 dbsnp gnomad3 clinvar`

Create folder for this project:

`$ mkdir Project_5`

`$ cd Project_5`

Install plink 1.9:

`$ mkdir tools && cd tools`

`$ wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20231211.zip`

`$ unzip plink_linux_x86_64_20231211.zip -d plink`

`$ rm plink_linux_x86_64_20231211.zip`

`$ cd ..`

---

## **Data processing**

### **Downloading**

Download manually raw 23andMe data from [here](https://drive.google.com/file/d/1QJkwJe5Xl_jSVpqdTSNXP7sqlYfI666j/view) and Genotek data from [here](https://drive.google.com/file/d/1piKi0FxVao-IvG81TAXhDKlHXvHE6uTm/view)

`$ mkdir data`

`$ mv ~/Downloads/genotek.vcf ~/path/to/dir/data/`  # Genotek

`$ mv ~/Downloads/SNP_raw_v4_Full_20170514175358.txt.zip ~/path/to/dir/data/`  # 23andME

`$ cd ~/path/to/dir/data/`

`$ unzip rm SNP_raw_v4_Full_20170514175358.txt.zip`

`$ rm SNP_raw_v4_Full_20170514175358.txt.zip`

`$ cd ..`

### **Preparing for annotation tools**

#### **plink**

Remove all SNPs corresponding to deletions and insertions to make the file compatible with annotation tools

`$ mkdir -p results/plink`

```bash
$ tools/plink/plink \
--23file data/SNP_raw_v4_Full_20170514175358.txt \
--recode vcf \
--out results/plink/snps_clean \
--output-chr MT \
--snps-only just-acgt
```

The resulting file contains all the analyzed SNPs, and we are interested only in **variable positions**

#### **open-cravat**


`$ oc gui `

With gui at the store we can install module to use 23andMe .txt file as input without any conversion


`$ oc run SNP_raw_v4_Full_20170514175358.txt -l hg19 -t excel`

---

## **Data analyzing**

### **Origin and haplogroups**

Web-tool [mthap](https://dna.jameslick.com/mthap/)

Choose raw 23andMe file: `SNP_raw_v4_Full_20170514175358.txt`

Interpret the results ([FAQ](https://dna.jameslick.com/mthap/FAQ.html)):

- "Match" is a marker for which you match what is required for this haplogroup.
- "Mismatch" is a marker required for the haplogroup for which you have a different test result. This can indicate either an inaccurate test result, a reversion for the marker or that you are a poor match for the haplogroup. It may also indicate that you are "in between" two different haplogroups.
- An "Extra" is a marker you have that is not part of the defining markers list for the haplogroup. These could be private mutations, an indication of a poor match, or an inaccurate test result.
- "No-Call" means that the marker was tested but, for whatever reason, it was not possible to determine the result.
- "Untested" means that your testing service did not test this position. Usually there are enough other markers to determine a match, but a long list of Untested markers reduces the accuracy of the match.

### **Eye color**

For eye colour identification we used this [article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3694299/):

![](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3694299/bin/CroatMedJ_54_0248-F2.jpg)

`$ cat results/plink/snps_clean.vcf | grep rs12913832` genotype A/G: → Not Blue

---

`$ cat results/plink/snps_clean.vcf | grep rs16891982` genotype C/G: → Not Brown

`$ cat results/plink/snps_clean.vcf | grep rs6119471` NA

---

`$ cat results/plink/snps_clean.vcf | grep rs12203592` genotype C/T: → Not Green

`$ cat results/plink/snps_clean.vcf | grep rs16891982` genotype C/G: → Not Green

### **Annotation of all SNPs, selection of clinically relevant ones**

#### **[VEP](https://grch37.ensembl.org/Homo_sapiens/Tools/VEP) (Variant Effect Predictor)**

Assembly: <u>GRCh37.p13</u>

Choose file: `snps_clean`

---

Command line equivalent of job:

`$ ./vep --af --af_gnomade --af_gnomadg --appris --biotype --buffer_size 500 --canonical --check_existing --distance 5000 --hgvs --mane --merged --numbers --plugin MPC,[path_to]/fordist_constraint_official_mpc_values.txt.gz --plugin LOEUF,file=[path_to]/loeuf_dataset_grch37.tsv.gz,match_by=gene --plugin CADD,[path_to]/CADD_GRCh37_1.6_whole_genome_SNVs.tsv.gz,[path_to]/CADD_GRCh37_1.6_InDels.tsv.gz --plugin SpliceAI,snv=[path_to]/spliceai_scores.masked.snv.hg19.vcf.gz,indel=[path_to]/spliceai_scores.masked.indel.hg19.vcf.gz --plugin Phenotypes,dir=[path_to]/,phenotype_feature=1,exclude_sources=COSMIC&HGMD-PUBLIC&Cancer_Gene_Census --plugin AncestralAllele,[path_to]/homo_sapiens_ancestor_GRCh37_e75.fa.gz --plugin IntAct,mapping_file=[path_to]/mutation_gc_map.txt.gz,mutation_file=[path_to]/mutations.tsv,pmid=1 --polyphen b --protein --pubmed --regulatory --show_ref_allele --sift b --species homo_sapiens --symbol --transcript_version --tsl --uploaded_allele --cache --input_file [input_data] --output_file [output_file] --port 3337`

##### **Download and move result files to working directory:**

`$ mkdir -p results/vec`

`$ mv ~/Downloads/<vec_result_file.vcf> ~/path/to/dir/results/vec`

`$ mv ~/Downloads/<vec_result_file.txt> ~/path/to/dir/results/vec`

##### **Filter pathogenic variants:**

`$ cat ~/path/to/dir/results/vec/<vec_result_file.vcf> | grep pathogenic | grep -wv benign > ~/path/to/dir/results/vec/pathogenic.vcf`

Example: `$ cat results/vec/j9zxJlHDXeHehG3F.vcf | grep pathogenic | grep -wv benign > results/vec/pathogenic.vcf`

**i5005436**

https://franklin.genoox.com/clinical-db/variant/snp/chr6-32008312-C-T?app=assessment-tools

---

**rs2004640**

https://franklin.genoox.com/clinical-db/variant/snp/chr7-128578301-G-T?app=assessment-tools


##### **Filter pathogenic variants w/o introns:**

`$ cat ~/path/to/dir/results/vec/<vec_result_file.vcf> | grep pathogenic | grep -wv benign | grep -wv intron_variant > ~/path/to/dir/results/vec/pathogenic_wo_intron.vcf`

Example:

**rs1024611**

https://franklin.genoox.com/clinical-db/variant/snp/chr17-32579788-A-G?app=assessment-tools

---

**i5005436** - already found

https://franklin.genoox.com/clinical-db/variant/snp/chr6-32008312-C-T?app=assessment-tools

#### **Processing Open-cravat output**

```{python}
import pandas as pd
input_file = '~/path/to/dir/results//SNP_raw_v4_Full_20170514175358.txt.xlsx'
sheet_name = 'Variant'
header_row = 1
df = pd.read_excel(input_file, sheet_name=sheet_name, header=header_row)
csv_file_path = '23andMe.csv'
df.to_csv(csv_file_path, index=False)

df_23andMe = pd.read_csv(r'23andMe.csv')
df_23andMe.shape()

# Filtered Global AF, Clinical Significance
df_23andMe = df_23andMe[((df_23andMe['Global AF'].isna()) | (df_23andMe['Global AF'] < 0.05)) & ((df_23andMe['Clinical Significance'].notnull()) & (df_23andMe['Clinical Significance'].apply(lambda val: all(val != s for s in ['Benign', 'Benign/Likely benign', 'Likely benign', 'Uncertain significance', 'drug response', 'not provided', 'Conflicting interpretations of pathogenicity', 'other']))))]
```

Then we can directly read about this varian on Franklin